In [2]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

warnings.filterwarnings('ignore')
sns.set_style()
plt.rcParams['figure.figsize'] = (12, 6)

# 1. Estimación de Insumos en el INT

En este cuadernillo se quieren estimar la cantidad de insumos que necesitará el INT para 2035 considerando su tasa de aumento en cada uno de los diagnósticos más relevantes. Los insumos que se quieren calcular son:

- Camas
- Pabellones
- Procedimientos más importantes (Resonancias, TAC Variadas, Estudios Histopatológicos - Biopsias, etc...)

In [6]:
egresos_torax_deis = pd.read_csv("../data/processed/egresos_torax_mes_y_dia.csv")
egresos_torax_grd = pd.read_excel("../data/processed/dias_estada_hospitalizados.xlsx")